In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('data/i5x25.csv')
df.head()

,tarefa,maq_1,maq_2,maq_3,maq_4,maq_5,peso,due_date
0,1,2,1,4,7,8,8,6
1,2,8,3,2,1,5,5,6
2,3,8,8,8,4,1,7,6
3,4,4,9,10,4,5,10,6
4,5,9,10,7,5,3,2,6
